# Project: Analyzing Wikipedia Pages

the goal of the project is to search for all occurences of a string in allof the files, providing a case-insensitive option to the search, and to refine the result by providing the specific locations of the files

## Introducing Wikipedia Data

listing the files in wiki folder

In [9]:
import os
file_names = os.listdir("wiki")
for i in range(3):
    print(file_names[i])

Ronald_McCaffer.html
IwakiIshikawa_Station.html
Communities_of_Tulu_Nadu.html


counting and displaying the number of files in the wiki folder

In [10]:
num_files = len(file_names)
print(num_files)

999


read the first line in the wiki folder and printing its contents

In [11]:
folder_name = "wiki"
file_name_first = file_names[0]
with open(os.path.join(folder_name, file_name_first)) as f:
    lines = [line for line in f.readlines()]
print(lines[1])

<html class="client-nojs" lang="en" dir="ltr">



## Adding the MapReduce Framework

the code for mapreduce

In [29]:
import math
import functools
from multiprocess import Pool

def make_chunks(data, num_chunks):
    chunk_size = math.ceil(len(data) / num_chunks)
    return [data[i:i+chunk_size] for i in range(0, len(data), chunk_size)]

def map_reduce(data, num_processes, mapper, reducer):
    chunks = make_chunks(data, num_processes)
    pool = Pool(num_processes)
    chunk_results = pool.map(mapper, chunks)
    return functools.reduce(reducer, chunk_results)

counting the total number of lines in all files

In [30]:
def map_line_count(file_names):
    total = 0
    for fn in file_names:
        with open(os.path.join("wiki", fn)) as f:
            total += len(f.readlines())
    return total
    
def reduce_line_count(count1, count2):
    return count1 + count2

results = map_reduce(file_names, 8, map_line_count, reduce_line_count)
print(results)

499797


## Grep Exact Match

using mapreduce to create a function that given a string, creates a dictionary where the keys are the file names and the values are lists with all line indexes that contain the string

In [45]:
def map_grep(file_names):
    results = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line for line in f.readlines()]
        for line_index, line in enumerate(lines):
            if target in line:
                if fn not in results:
                    results[fn] = []
                results[fn].append(line_index)
    return results

def reduce_grep(lines1, lines2):
    lines1.update(lines2)
    return lines1

def mapreduce_grep(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes,  map_grep, reduce_grep)

target = "data"
data_occurrences = mapreduce_grep("wiki", 8)


## Grep Case Insensitive

creating a function that creates a dictionary. the keys are the filenames and the values are lists with all indices that contain the given string

In [44]:
def map_grep_insensitive(file_names):
    results = {}
    for fn in file_names:
        with open(fn) as f:
            lines = [line.lower() for line in f.readlines()]
        for line_index, line in enumerate(lines):
            if target.lower() in line:
                if fn not in results:
                    results[fn] = []
                results[fn].append(line_index)
    return results

def mapreduce_grep_insensitive(path, num_processes):
    file_names = [os.path.join(path, fn) for fn in os.listdir(path)]
    return map_reduce(file_names, num_processes,  map_grep_insensitive, reduce_grep)



finding all occurences of the string "data"

In [46]:
target = "data"
new_data_occurrences = mapreduce_grep_insensitive("wiki", 8)


## Checking the Implementation

storing the results of the search for string "data" with both versions of the algorithms into variables

In [47]:
for fn in new_data_occurrences:
    if fn not in data_occurrences:
        print("Found {} new matches on file {}".format(len(new_data_occurrences[fn]), fn))
    elif len(new_data_occurrences[fn]) > len(data_occurrences[fn]):
        print("Found {} new matches on file {}".format(len(new_data_occurrences[fn]) - len(data_occurrences[fn]), fn))

Found 1 new matches on file wiki/Tim_Spencer_(singer).html
Found 1 new matches on file wiki/Embraer_Unidade_GaviC3A3o_Peixoto_Airport.html
Found 1 new matches on file wiki/Lower_Blackburn_Grade_Bridge.html
Found 1 new matches on file wiki/HD_90156.html
Found 2 new matches on file wiki/Kate_Harwood.html
Found 1 new matches on file wiki/Doumanaba.html
Found 6 new matches on file wiki/Dragnet_(franchise).html
Found 1 new matches on file wiki/Boardman_Township_Mahoning_County_Ohio.html
Found 1 new matches on file wiki/Brownfield_(software_development).html
Found 1 new matches on file wiki/Kattukukke.html
Found 2 new matches on file wiki/The_Audacity_to_Podcast.html
Found 1 new matches on file wiki/Derek_Acorah.html
Found 1 new matches on file wiki/Lis_LC3B8wert.html
Found 1 new matches on file wiki/KMTZ.html
Found 1 new matches on file wiki/Battle_of_Wattignies.html
Found 1 new matches on file wiki/Manhattan_Murder_Mystery.html
Found 2 new matches on file wiki/Shpolskii_matrix.html
Found 1

modifying the mapper function so that it returns a list of pairs with all occurences